In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

%matplotlib inline

In [2]:
train_df = pd.read_csv("train.csv", low_memory=False)
test_df = pd.read_csv("test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [3]:
data.columns

Index(['id', 'days_to_today', 'año', 'antiguedad',
       'centroscomercialescercanos', 'escuelascercanas', 'habitaciones',
       'garages', 'banos', 'anio', 'Apartamento', 'Bodega comercial', 'Casa',
       'Casa en condominio', 'Casa uso de suelo', 'Departamento Compartido',
       'Duplex', 'Edificio', 'Garage', 'Hospedaje', 'Huerta',
       'Inmuebles productivos urbanos', 'Local Comercial',
       'Local en centro comercial', 'Lote', 'Nave industrial',
       'Oficina comercial', 'Otros', 'Quinta Vacacional', 'Rancho', 'Terreno',
       'Terreno comercial', 'Terreno industrial', 'Villa', 'gimnasio',
       'usosmultiples', 'piscina', 'qty_of_extras', 'metroscubiertos',
       'metrostotales', 'popular_desc_1', 'popular_desc_2', 'popular_desc_3',
       'popular_desc_4', 'popular_desc_5', 'popular_desc_6', 'popular_desc_7',
       'popular_desc_8', 'popular_desc_9', 'popular_desc_10',
       'popular_desc_11', 'popular_desc_12', 'popular_desc_13', 'idzona',
       'lat', 'lng', 'a

In [4]:
data.rename(columns={'año':'ano', 'Yucatán':'Yucatan', 'Querétaro':'Queretaro','Michoacán':'Michoacan',
                     'Nuevo León':'Nuevo Leon',
                     'San luis Potosí':'San luis Potosi', 'Edo. de México':'Edo. de Mexico'}, inplace=True)

In [5]:
data = data[['id','days_to_today',
 'antiguedad',
 'habitaciones',
 'garages',
 'banos',
 'Apartamento',
 'Casa',
 'Edificio',
 'Terreno',
 'gimnasio',
 'qty_of_extras',
 'metroscubiertos',
 'metrostotales',
 'popular_desc_1',
 'popular_desc_2',
 'popular_desc_7',
 'popular_desc_13',
 'idzona',
 'lat',
 'lng',
 'Coahuila',
 'Distrito Federal',
 'Edo. de Mexico',
 'Jalisco',
 'Nuevo Leon',
 'Yucatan',
 'dolar',
 'habxmetros',
 'habxmetroscub',
 'banosxmetroscub',
 'banosxhab']]

In [6]:
train_df = train_df[['id','precio']]
train_df['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [17]:
features = pd.merge(train_df, data, on='id', how='inner')
features = features.fillna(0)
features['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [18]:
labels = features['precio']
features = features.drop(['id','precio'], axis=1)
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features.fillna(0, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  180000 Test:  60000


In [19]:
train_data=lgb.Dataset(x_train, label=y_train)

In [20]:
lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=300, 
                              boosting_type= 'gbdt', colsample_bytree= 0.65, 
                              learning_rate= 0.07, num_leaves= 27, random_state= 501, 
                              subsample= 0.7)


In [22]:
lgb_model.fit(x_train, y_train)
y_LGB_predict = lgb_model.predict(x_test)


In [25]:
LGB_model_error = sqrt(mean_absolute_error(y_test,y_LGB_predict))
print(LGB_model_error)

784.0816589729961


In [28]:
XGB_model = xgb.XGBRegressor(learning_rate=0.3, max_depth=6, n_estimators=500)
XGB_model.fit(x_train, y_train)
y_XGB_predict = XGB_model.predict(x_test)


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:43:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [29]:
XGB_model_error = sqrt(mean_absolute_error(y_test,y_XGB_predict))
print(XGB_model_error)

750.7036229016405


In [37]:
ensembled_prediction = (0.8*(y_XGB_predict))+(0.2*(y_LGB_predict))
ensembled_prediction_error = sqrt(mean_absolute_error((y_test), ensembled_prediction))

print(ensembled_prediction_error)

747.0190398539756


array([inf, inf, inf, ..., inf, inf, inf])

In [38]:
set_test = pd.merge(data, test_df[['id']],on = "id", how = "inner")
ids = set_test["id"]
set_test = set_test.drop(columns=['id'])

In [39]:
set_test = set_test.fillna(0)
set_test.replace([np.inf, -np.inf], np.nan, inplace=True)
set_test.fillna(0, inplace=True)


In [40]:
set_test.head()

,days_to_today,antiguedad,habitaciones,garages,banos,Apartamento,Casa,Edificio,Terreno,gimnasio,...,Distrito Federal,Edo. de Mexico,Jalisco,Nuevo Leon,Yucatan,dolar,habxmetros,habxmetroscub,banosxmetroscub,banosxhab
0,2324,29.0,3.0,0.0,4.0,0,1,0,0,0.0,...,1,0,0,0,0,12.7325,0.000000,0.010000,0.013333,0.0
1,1498,0.0,1.0,1.0,1.0,1,0,0,0,0.0,...,0,0,0,0,1,16.5000,0.014925,0.014925,0.014925,1.0
2,1645,0.0,2.0,1.0,2.0,1,0,0,0,0.0,...,1,0,0,0,0,15.3775,0.020000,0.022989,0.022989,2.0
3,1703,2.0,2.0,2.0,2.0,1,0,0,0,0.0,...,0,0,0,0,0,15.3488,0.023256,0.023256,0.023256,1.0
4,2298,10.0,2.0,1.0,1.0,0,1,0,0,0.0,...,0,1,0,0,0,13.3803,0.026316,0.025000,0.012500,2.0


In [51]:
y_XGB_predict = (XGB_model.predict(set_test))
y_LGB_predict = (lgb_model.predict(set_test))


In [52]:
ensembled_prediction = (0.8*y_XGB_predict)+(0.2*y_LGB_predict)


In [53]:
df_kaggel = pd.DataFrame(ensembled_prediction)
df_kaggel.head()

,0
0,6.945443e+06
1,1.123755e+06
2,2.323822e+06
3,1.291460e+06
4,6.725496e+05


In [54]:
df_kaggel = pd.DataFrame(df_kaggel)                               
df_kaggel["id"] = ids
df_kaggel.rename(columns = {0: 'target'},inplace = True) 
df_kaggel = df_kaggel[['id','target']]

In [55]:
df_kaggel.head()

,id,target
0,4941,6.945443e+06
1,51775,1.123755e+06
2,115253,2.323822e+06
3,299321,1.291460e+06
4,173570,6.725496e+05


In [56]:
df_kaggel.to_csv('ResultadosLightGBMyXGB.csv',index=False)

In [57]:
df_kaggel.shape

(60000, 2)

In [58]:
df_kaggel.head()

,id,target
0,4941,6.945443e+06
1,51775,1.123755e+06
2,115253,2.323822e+06
3,299321,1.291460e+06
4,173570,6.725496e+05
